In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

c:\python\py\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Steve\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [ ]:
######importations des modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import warnings
warnings.filterwarnings("ignore")


In [3]:
### charge des données et fusion des datasets
print("🔹 Chargement des données...")

def load_data(path):
    #charger les données
    customers = pd.read_csv(path + "/olist_customers_dataset.csv")
    geolocation = pd.read_csv(path + "/olist_geolocation_dataset.csv")
    order_items = pd.read_csv(path + "/olist_order_items_dataset.csv")
    order_payments = pd.read_csv(path + "/olist_order_payments_dataset.csv")
    order_reviews = pd.read_csv(path + "/olist_order_reviews_dataset.csv")
    orders = pd.read_csv(path + "/olist_orders_dataset.csv")
    products = pd.read_csv(path + "/olist_products_dataset.csv")
    sellers = pd.read_csv(path + "/olist_sellers_dataset.csv")
    products_translation = pd.read_csv(path + "/product_category_name_translation.csv")

    

    # Fusionner les datasets
    df = orders.merge(customers, on='customer_id', how='left')
    df = df.merge(order_items, on='order_id', how='left')
    df = df.merge(order_payments, on='order_id', how='left')
    df = df.merge(order_reviews, on='order_id', how='left')
    df = df.merge(products, on='product_id', how='left')
    df = df.merge(sellers, on='seller_id', how='left')
    df = df.merge(products_translation, on='product_category_name', how='left')

     # ⚠️ Gestion du dataset geolocation (moyenne par zip_code_prefix)
    geo_mean = geolocation.groupby("geolocation_zip_code_prefix")[["geolocation_lat", "geolocation_lng"]].mean().reset_index()
    df = df.merge(geo_mean, left_on="customer_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left")
    df = df.merge(geo_mean, left_on="seller_zip_code_prefix", right_on="geolocation_zip_code_prefix", how="left", suffixes=('_customer', '_seller'))
    
    return df

🔹 Chargement des données...


In [4]:
# affi geolacation des données
df = load_data(path)
print("🔹 Données chargées avec succès.")
# Afficher les premières lignes du DataFrame
df.head()


🔹 Données chargées avec succès.


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,geolocation_zip_code_prefix_customer,geolocation_lat_customer,geolocation_lng_customer,geolocation_zip_code_prefix_seller,geolocation_lat_seller,geolocation_lng_seller
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,9350.0,maua,SP,housewares,3149.0,-23.576983,-46.587161,9350.0,-23.680729,-46.444238
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,9350.0,maua,SP,housewares,3149.0,-23.576983,-46.587161,9350.0,-23.680729,-46.444238
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,9350.0,maua,SP,housewares,3149.0,-23.576983,-46.587161,9350.0,-23.680729,-46.444238
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,31570.0,belo horizonte,SP,perfumery,47813.0,-12.177924,-44.660711,31570.0,-19.807681,-43.980427
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,14840.0,guariba,SP,auto,75265.0,-16.745150,-48.514783,14840.0,-21.363502,-48.229601


In [5]:
df.isnull().sum()  # Vérifier les données manquantes


order_id                                     0
customer_id                                  0
order_status                                 0
order_purchase_timestamp                     0
order_approved_at                          177
order_delivered_carrier_date              2086
order_delivered_customer_date             3421
order_estimated_delivery_date                0
customer_unique_id                           0
customer_zip_code_prefix                     0
customer_city                                0
customer_state                               0
order_item_id                              833
product_id                                 833
seller_id                                  833
shipping_limit_date                        833
price                                      833
freight_value                              833
payment_sequential                           3
payment_type                                 3
payment_installments                         3
payment_value

In [6]:
def cleaning_data(df):
    # Supprimer toutes les lignes avec des valeurs manquantes dans le DataFrame
    df.dropna(inplace=True)
    # Supprimer les doublons
    df.drop_duplicates(inplace=True)
    

    # unif
    return df

In [7]:
df = cleaning_data(df)
df.count() # Vérifier à nouveau les données manquantes

order_id                                11530
customer_id                             11530
order_status                            11530
order_purchase_timestamp                11530
order_approved_at                       11530
order_delivered_carrier_date            11530
order_delivered_customer_date           11530
order_estimated_delivery_date           11530
customer_unique_id                      11530
customer_zip_code_prefix                11530
customer_city                           11530
customer_state                          11530
order_item_id                           11530
product_id                              11530
seller_id                               11530
shipping_limit_date                     11530
price                                   11530
freight_value                           11530
payment_sequential                      11530
payment_type                            11530
payment_installments                    11530
payment_value                     

In [ ]:
from geopy.distance import geodesic

def build_dataframe(df):
    # 1. Convertir les dates
    date_col = [
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ]
    for col in date_col:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # délai de livraison (jours)
    df["delivery_time"] = (df["order_delivered_customer_date"] - df["order_purchase_timestamp"]).dt.days

    # 2. Order_items → prix total et nb d’articles
    df["total_price"] = df.groupby("order_id")["price"].transform("sum")
    df["total_items"] = df.groupby("order_id")["order_item_id"].transform("count")

    # 3. Order_payments → montant total et nb paiements
    df["total_payment"] = df.groupby("order_id")["payment_value"].transform("sum")
    df["payment_count"] = df.groupby("order_id")["payment_type"].transform("count") # nombre de paiements

    # 4. Reviews → note moyenne
    df["average_review_score"] = df.groupby("order_id")["review_score"].transform("mean")

    # 5. Produits → catégorie
    df["product_category_name"] = df["product_category_name"].astype("category")

    # 6. Distance client-vendeur
    def safe_distance(row): 
        if pd.isna(row["geolocation_lat_customer"]) or pd.isna(row["geolocation_lng_customer"]) \
        or pd.isna(row["geolocation_lat_seller"]) or pd.isna(row["geolocation_lng_seller"]):
            return None
        return geodesic(
            (row["geolocation_lat_customer"], row["geolocation_lng_customer"]),
            (row["geolocation_lat_seller"], row["geolocation_lng_seller"])
        ).km

    df["distance"] = df.apply(safe_distance, axis=1)

    # 7. Créer la target
    df["satisfaction"] = (df["review_score"] >= 4).astype(int)

    # 8. Supprimer les colonnes inutiles
    columns_to_drop = [
        "order_id", "customer_id", "order_item_id", "payment_sequential", 
        "payment_installments", "review_id", "product_id",
        "seller_id", "product_category_name_english",
        "order_purchase_timestamp", "order_approved_at",
        "order_delivered_carrier_date", "order_delivered_customer_date",
        "order_estimated_delivery_date",
        "review_comment_title", "review_comment_message",
        "review_creation_date", "review_answer_timestamp",
        "geolocation_lat_customer", "geolocation_lng_customer",
        "geolocation_lat_seller", "geolocation_lng_seller"
    ]
    df.drop(columns=columns_to_drop, inplace=True, errors="ignore")

    # 9. Sélection des features et de la target
    features = ["total_price", "total_items", "total_payment", 
                "payment_count", "distance", "delivery_time"]
    
    X = df[features]
    y = df["satisfaction"]

    return X, y


In [9]:
# afficher les données traitées
X, y = build_dataframe(df)



In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 , shuffle=True)

In [11]:
X_train

,total_price,total_items,total_payment,payment_count,distance,delivery_time
16930,169.75,2,469.44,2,592.043188,12
73623,199.00,1,209.37,1,300.334300,18
64770,320.00,4,93.92,4,361.877233,11
73364,19.90,1,43.18,1,739.786297,6
18984,169.99,1,193.98,1,478.086415,11
...,...,...,...,...,...,...
116300,299.99,1,339.67,1,7.854652,6
54496,192.90,1,212.05,1,341.736118,6
56470,144.90,1,161.01,1,361.491657,4
9003,105.00,1,114.67,1,75.766503,3


In [12]:
### pipline de traitement des données
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

def create_pipeline():
    # Définir les colonnes numériques et catégorielles
    numeric_features = ["total_price", "total_items", "total_payment", 
                        "payment_count", "distance", "delivery_time"]
    

    # Créer le préprocesseur pour les colonnes numériques
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    # utiliser ColumnTransformer pour appliquer les transformations
    preprocessor = ColumnTransformer(   
        transformers=[
            ("num", numeric_transformer, numeric_features)
        ],
        remainder="passthrough"  # Conserver les colonnes non spécifiées
    )

    return preprocessor

In [13]:
## pipeline de traitement des données
def model_pipeline(model):
    # Créer le pipeline de traitement des données
    preprocessor = create_pipeline()

    # Créer le pipeline complet avec le modèle
    pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", model)
    ])

    return pipeline

In [ ]:
# random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Définir le modèle
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 2. Créer le pipeline
pipeline = model_pipeline(model)

# 3. Entraîner le pipeline sur le train
pipeline.fit(X_train, y_train)

# 4. Prédire sur le test
y_pred = pipeline.predict(X_test)

# 5. Évaluer la performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8061578490893322

Classification report:
               precision    recall  f1-score   support

           0       0.76      0.54      0.63       711
           1       0.82      0.92      0.87      1595

    accuracy                           0.81      2306
   macro avg       0.79      0.73      0.75      2306
weighted avg       0.80      0.81      0.80      2306



In [15]:
# xgboost regressor
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
pipeline = model_pipeline(model)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Afficher les résultats
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7983521248915871

Classification report:
               precision    recall  f1-score   support

           0       0.77      0.49      0.60       711
           1       0.81      0.93      0.87      1595

    accuracy                           0.80      2306
   macro avg       0.79      0.71      0.73      2306
weighted avg       0.79      0.80      0.78      2306



In [16]:
# enregistrement du modèle
import joblib
joblib.dump(pipeline, "rf_pipeline.pkl")

['rf_pipeline.pkl']

In [ ]:

# Charger le pipeline 
predict= joblib.load("rf_pipeline.pkl")

# Nouvelles données
new_data = pd.DataFrame({
    "total_price": [150.0],
    "total_items": [3],
    "total_payment": [155.0],
    "payment_count": [1],
    "distance": [100.5],
    "delivery_time": [117]
})

# Faire la prédiction
predictions = pipeline.predict(new_data)

print("Predictions:", predictions)



Predictions: [1]
